In [18]:
import numpy as np
import tensorflow as tf

In [19]:
"""
在训练模型的时候， 我们可以将模型训练放在GPU上去跑，这样可以提高训练速度
但是训练之前的数据处理（读取、清洗、特征工程等操作）阶段，是需要CPU来处理的， 这个阶段我们如果慢下来的话，
那么整个过程我们依旧会很慢很慢。
为了解决这个问题，Tensorflow官网提供了一个数据结构TFRecords,即一次性将数据存储成该数据结构，
后续每次读取的时候只要加载这个结构的数据即可，读取该结构的数据时，速度会很快
TFRecords 表示了一整个数据集
由tf.train.Example构成 （tf.train.Example表示的是一个样本数据）
构造一个TFRecords文件，需要构建若干个tf.train.Example，然后将若干个tf.train.Example写入到一个文件A中，
文件A就是一个TFRecords数据集了

通常情况下tf.train.Example支持以下几种数据格式:
tf.train.BytesList: 可以使用的类型包括string和byte
tf.train.FloatList: 可以使用的类型包括float和dubbo
tf.train.Int64List: 可以使用的类型包括enum、bool、int32、uint32、int64
"""

'\n在训练模型的时候， 我们可以将模型训练放在GPU上去跑，这样可以提高训练速度\n但是训练之前的数据处理（读取、清洗、特征工程等操作）阶段，是需要CPU来处理的， 这个阶段我们如果慢下来的话，\n那么整个过程我们依旧会很慢很慢。\n为了解决这个问题，Tensorflow官网提供了一个数据结构TFRecords,即一次性将数据存储成该数据结构，\n后续每次读取的时候只要加载这个结构的数据即可，读取该结构的数据时，速度会很快\nTFRecords 表示了一整个数据集\n由tf.train.Example构成 （tf.train.Example表示的是一个样本数据）\n构造一个TFRecords文件，需要构建若干个tf.train.Example，然后将若干个tf.train.Example写入到一个文件A中，\n文件A就是一个TFRecords数据集了\n\n通常情况下tf.train.Example支持以下几种数据格式:\ntf.train.BytesList: 可以使用的类型包括string和byte\ntf.train.FloatList: 可以使用的类型包括float和dubbo\ntf.train.Int64List: 可以使用的类型包括enum、bool、int32、uint32、int64\n'

In [20]:
# 转化实例
def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [21]:
def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

In [22]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [24]:
# 测试上面的三个函数
print(_bytes_feature(b'test_string'))
print(_bytes_feature('test_string'.encode('utf8')))

# 浮点型
print(_float_feature(np.exp(1)))

# 整数型
print(_int64_feature(2))

bytes_list {
  value: "test_string"
}

bytes_list {
  value: "test_string"
}

float_list {
  value: 2.7182817459106445
}

int64_list {
  value: 2
}



In [25]:
### TFRecords制作方法
# 创建tf.train.Example
def serialize_example(feature0, feature1, feature2, feature3):
    # 转换成相应类型
    feature = {
        'feature0': _int64_feature(feature0),
        'feature1': _int64_feature(feature1),
        'feature2': _bytes_feature(feature2),
        'feature3': _float_feature(feature3)
    }
    example = tf.train.Example(features=tf.train.Features(feature=feature))
    return example.SerializeToString()

In [27]:
# 数据量
n_observations = int(1e4)

# Boolean features
feature0 = np.random.choice([False, True], n_observations)
# Integer
feature1 = np.random.randint(0, 5, n_observations)
# String
strings = np.array([b'cat', b'dog', b'chicken', b'horse', b'goat'])
feature2 = strings[feature1]
# Float
feature3 = np.random.randn(n_observations)

In [35]:
# 写入文件
filename='tfrecord-1'
with tf.io.TFRecordWriter(filename) as writer:
    for i in range(n_observations):
        example = serialize_example(feature0[i], feature1[i], feature2[i], feature3[i])
        writer.write(example)

C:\Users\songc\AppData\Local\Temp/ipykernel_16268/3454670613.py:2: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


In [38]:
# 读取TFRecords文件
filenames = [filename]
# 读取文件
row_dataset = tf.data.TFRecordDataset(filenames)
row_dataset

<TFRecordDatasetV2 element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>